<p style="text-align: center;">
    openai / whisper - Speech Recognition 
</p> 
<p style="text-align: center;"> 
    Added Support : webVTT, SRTT  
</p> 

In [1]:
import os
import whisper
import time
import sys

In [2]:
def transcribe(model_name: str, audio_path: str):
    model = whisper.load_model(model_name)
    result = model.transcribe(audio_path)
    transcription = result["segments"]
    return transcription

In [3]:
def first_three_decimal_places(time: float): 
    float_to_fixed_3_decimal_places = str("%.3f" %  time) 
    removed_int = float_to_fixed_3_decimal_places[float_to_fixed_3_decimal_places.find('.'):]
    removed_decimal_point = removed_int[1:]  
    return removed_decimal_point

In [4]:
def transcription(each_transcription):
    output = {}
    output["id"] = int(each_transcription["id"])
    output["start_time"] = time.strftime('%H:%M:%S', time.gmtime(each_transcription["start"]))  
    output["start_time_decimal"] = first_three_decimal_places(each_transcription["start"])
    output["end_time"] = time.strftime('%H:%M:%S', time.gmtime(each_transcription["end"])) 
    output["end_time_decimal"] = first_three_decimal_places(each_transcription["end"])
    output["text"] = each_transcription["text"].lstrip(" ")  
    return output

In [5]:
def checkIfFileExists(fileName: str, fileFormat: str):
    if not os.path.exists(f"{fileName}.{fileFormat}"):
        with open(f"{fileName}.{fileFormat}", 'w'): pass

In [6]:
def webVTT(webVTT_Name: str, whole_transcription: list): 
    checkIfFileExists(webVTT_Name, "vtt")
    with open(f"{webVTT_Name}.vtt", 'w') as f:
        f.write("WEBVTT")
        for x in range(len(whole_transcription)):
            each_transcription = transcription(whole_transcription[x])
            start_time = each_transcription["start_time"]
            start_time_decimal = each_transcription["start_time_decimal"]
            end_time = each_transcription["end_time"]
            end_time_decimal = each_transcription["end_time_decimal"]
            text = each_transcription["text"]      
            f.write("\n\n")
            f.write(f"{start_time}.{start_time_decimal} --> {end_time}.{end_time_decimal}")
            f.write("\n")
            f.write(f"{text}")
    return os.path.abspath(f"{webVTT_Name}.vtt")

In [7]:
def SRT(SRT_Name: str, whole_transcription: list):
    checkIfFileExists(SRT_Name, "srt")
    with open(f"{SRT_Name}.srt", 'w') as f:
        for x in range(len(whole_transcription)): 
            each_transcription = transcription(whole_transcription[x])
            id = each_transcription["id"]
            start_time = each_transcription["start_time"]
            start_time_decimal = each_transcription["start_time_decimal"]
            end_time = each_transcription["end_time"]
            end_time_decimal = each_transcription["end_time_decimal"]
            text = each_transcription["text"]  
            f.write(f"{id + 1}")
            f.write("\n")
            f.write(f"{start_time},{start_time_decimal} --> {end_time},{end_time_decimal}")
            f.write("\n")
            f.write(f"{text}")  
            if not x + 1 == len(whole_transcription):
                f.write("\n\n")
    return os.path.abspath(f"{SRT_Name}.srt")


Output

In [10]:
def output(model_name: str, audio_path: str, SRT_bool: bool, webVTT_bool: bool):
    if not os.path.exists(f"{audio_path}"):
        return "Invalid Audio Path"
    if (SRT_bool or webVTT_bool) != True: 
        return "Select At Least One Subtitle Format"
        
    whole_transcription = transcribe(model_name, audio_path)  

    Transcripts = []
    if SRT_bool == True: 
        Transcripts.append(SRT(audio_path, whole_transcription)) 
    if webVTT_bool == True: 
        Transcripts.append(webVTT(audio_path, whole_transcription))
    return Transcripts

In [ ]:
if __name__ == '__main__':
    print(output("base", str(sys.argv[1]), True, True))
    sys.stdout.flush()